<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Example-1" data-toc-modified-id="Example-1-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Example 1</a></span><ul class="toc-item"><li><span><a href="#Document-Encoding" data-toc-modified-id="Document-Encoding-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Document Encoding</a></span><ul class="toc-item"><li><span><a href="#One-hot" data-toc-modified-id="One-hot-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>One hot</a></span></li><li><span><a href="#Tokenizer" data-toc-modified-id="Tokenizer-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Tokenizer</a></span></li></ul></li><li><span><a href="#Padding" data-toc-modified-id="Padding-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Padding</a></span><ul class="toc-item"><li><span><a href="#One-Hot" data-toc-modified-id="One-Hot-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>One Hot</a></span></li><li><span><a href="#Tokenizer" data-toc-modified-id="Tokenizer-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Tokenizer</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#Embedding" data-toc-modified-id="Embedding-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Embedding</a></span></li><li><span><a href="#LSTM-without-Embedding" data-toc-modified-id="LSTM-without-Embedding-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>LSTM without Embedding</a></span></li></ul></li><li><span><a href="#Complete" data-toc-modified-id="Complete-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Complete</a></span></li></ul></li><li><span><a href="#Not-Using-Embedding" data-toc-modified-id="Not-Using-Embedding-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Not Using Embedding</a></span></li><li><span><a href="#Tokenizer" data-toc-modified-id="Tokenizer-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Tokenizer</a></span></li><li><span><a href="#GloVe-Embedding" data-toc-modified-id="GloVe-Embedding-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>GloVe Embedding</a></span><ul class="toc-item"><li><span><a href="#Complete" data-toc-modified-id="Complete-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Complete</a></span></li></ul></li></ul></div>

# Keras Word Embedding 

This notebook demonstrates ideas about word embedding, tokenizer, and LSTM. 

A word embedding is a class of approaches for representing words and documents using a dense vector representation.

Original tutorial uses `one_hot` function, but I implement this code using the `Tokenizer`. 

- https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
- https://keras.io/layers/embeddings/

## Example 1

In [7]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import LSTM

print('done')

done


In [2]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

### Document Encoding

#### One hot

`**one_hot**`

**Returns** List of integers in [1, n]. Each integer encodes a word (unicity non-guaranteed).

https://keras.io/preprocessing/text/#one_hot

In [12]:
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[12, 23], [32, 14], [34, 25], [29, 14], [32], [1], [2, 25], [28, 32], [2, 14], [24, 23, 23, 47]]


Note that Word Good and Weak are both encoded as integer 32. This is because unicity is non-guaranteed by one_hot function.

#### Tokenizer

Use this instead of One Hot

In [3]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)

# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


### Padding

#### One Hot

In [11]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[12 23  0  0]
 [32 14  0  0]
 [34 25  0  0]
 [29 14  0  0]
 [32  0  0  0]
 [ 1  0  0  0]
 [ 2 25  0  0]
 [28 32  0  0]
 [ 2 14  0  0]
 [24 23 23 47]]


#### Tokenizer

In [4]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


### Model

#### Embedding

In [30]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 89.999998


In [42]:
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


#### LSTM without Embedding

Not too sure how to implement this yet. 

In [9]:
# create the model
embedding_vecor_length = 32

model = Sequential()
# model.add(Embedding(vocab_size, embedding_vecor_length, input_length=max_length))
model.add(LSTM(10))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
print(model.summary())

# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

TypeError: add() got an unexpected keyword argument 'inputs'

### Complete

In [16]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

[[12, 23], [32, 14], [34, 25], [29, 14], [32], [1], [2, 25], [28, 32], [2, 14], [24, 23, 23, 47]]
[[12 23  0  0]
 [32 14  0  0]
 [34 25  0  0]
 [29 14  0  0]
 [32  0  0  0]
 [ 1  0  0  0]
 [ 2 25  0  0]
 [28 32  0  0]
 [ 2 14  0  0]
 [24 23 23 47]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten_2 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 89.999998


## Not Using Embedding

## Tokenizer

https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/

In [17]:
from keras.preprocessing.text import Tokenizer
# define 5 documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!']
# create the tokenizer
t = Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(docs)

In [19]:
# summarize what was learned
print(t.word_counts)

OrderedDict([('well', 1), ('done', 1), ('good', 1), ('work', 2), ('great', 1), ('effort', 1), ('nice', 1), ('excellent', 1)])


In [20]:
print(t.document_count)

5


In [21]:
print(t.word_index)

{'work': 1, 'well': 2, 'done': 3, 'good': 4, 'great': 5, 'effort': 6, 'nice': 7, 'excellent': 8}


In [22]:
print(t.word_docs)

{'done': 1, 'well': 1, 'good': 1, 'work': 2, 'effort': 1, 'great': 1, 'nice': 1, 'excellent': 1}


In [23]:
# integer encode documents
encoded_docs = t.texts_to_matrix(docs, mode='count')
print(encoded_docs)

[[0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## GloVe Embedding

In [25]:
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])

In [26]:
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

In [29]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)

[[6, 2], [3, 1], [7, 4], [8, 1], [9], [10], [5, 4], [11, 3], [5, 1], [12, 13, 2, 14]]


In [30]:
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[ 6  2  0  0]
 [ 3  1  0  0]
 [ 7  4  0  0]
 [ 8  1  0  0]
 [ 9  0  0  0]
 [10  0  0  0]
 [ 5  4  0  0]
 [11  3  0  0]
 [ 5  1  0  0]
 [12 13  2 14]]


In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [ ]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)

### Complete

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('../glove_data/glove.6B/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))